# Biomass Mapping with gridFIA

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mihiarc/fiatools/blob/main/tutorials/02_biomass_mapping_with_gridfia.ipynb)
[![FIAtools](https://img.shields.io/badge/FIAtools-Ecosystem-2E7D32)](https://fiatools.org)

This tutorial shows you how to create forest biomass and species diversity maps using **gridFIA** and USDA Forest Service BIGMAP data.

## What You'll Learn

- What BIGMAP data is and its capabilities
- How to download species-level biomass data
- How to calculate diversity indices (Shannon, Simpson)
- How to create publication-ready maps

## Prerequisites

- Python 3.9 or higher
- Basic familiarity with geospatial concepts

**Note:** This tutorial involves downloading ~100-500MB of raster data from the USDA Forest Service. Downloads may take 5-15 minutes depending on the area selected.

---

## 1. What is BIGMAP?

BIGMAP (Biomass Informed by Geospatial Mapping and Allometric Product) is a USDA Forest Service dataset that provides:

- **30-meter resolution** biomass maps for the contiguous US
- **327 tree species** with individual biomass layers
- **2018 base year** derived from Landsat imagery and FIA plot data
- **Tons per acre** biomass units for each pixel

### Why gridFIA?

gridFIA provides:

- **Simple Python API** for downloading and analyzing BIGMAP data
- **Zarr storage** for efficient multi-species analysis
- **Built-in diversity calculations** (Shannon, Simpson, species richness)
- **Publication-ready visualizations**

## 2. Installation

Install gridFIA from PyPI:

In [ ]:
# Install gridFIA
!pip install -q gridfia

## 3. Quick Start: The GridFIA API

gridFIA uses a single entry point - the `GridFIA` class:

In [ ]:
from gridfia import GridFIA

# Initialize the API
api = GridFIA()

# List available species (this queries the BIGMAP service)
species = api.list_species()
print(f"Available species: {len(species)}")

# Show first few species
for s in species[:5]:
    print(f"  {s.species_code}: {s.common_name}")

## 4. Downloading Species Data

Download biomass data for specific species in a geographic area. We'll use a small county to keep the download quick.

**Note:** Downloads from the USDA Forest Service may take several minutes.

In [ ]:
# Download loblolly pine and red maple for a small county in North Carolina
# This may take 5-10 minutes depending on your connection

files = api.download_species(
    output_dir="data/wake_county",
    species_codes=["0131", "0316"],  # Loblolly pine, Red maple
    state="North Carolina",
    county="Wake"
)

print(f"Downloaded {len(files)} species files:")
for f in files:
    print(f"  {f.name}")

### Common Species Codes

| Code | Species | Region |
|------|---------|--------|
| 0131 | Loblolly pine | Southeast |
| 0121 | Longleaf pine | Southeast |
| 0202 | Douglas-fir | Pacific Northwest |
| 0316 | Red maple | Eastern US |
| 0802 | White oak | Eastern US |
| 0833 | Northern red oak | Eastern US |

Use `api.list_species()` to see all 327 available species.

## 5. Creating a Zarr Store

Combine downloaded rasters into a Zarr store for efficient analysis:

In [ ]:
# Create Zarr store from downloaded files
zarr_path = api.create_zarr(
    input_dir="data/wake_county",
    output_path="data/wake_county.zarr"
)

print(f"Created Zarr store: {zarr_path}")

### Why Zarr?

Zarr is a cloud-optimized format that:
- Stores multi-species data efficiently
- Enables lazy loading for large datasets
- Supports parallel processing
- Works with xarray for analysis

## 6. Calculating Total Biomass

Calculate total biomass from the downloaded species:

In [ ]:
# Calculate total biomass from downloaded species
results = api.calculate_metrics(
    zarr_path="data/wake_county.zarr",
    calculations=["total_biomass"]  # Sum of all species
)

# View results
print(f"Completed {len(results)} calculations:")
for result in results:
    print(f"  {result.name}: saved to {result.output_path}")
    if result.statistics:
        for stat, value in result.statistics.items():
            print(f"    {stat}: {value:.2f}")

### Note on Diversity Calculations

**Important:** Diversity indices (Shannon, Simpson, species richness) require data from **all species present in an area**, not just a few selected species. With only 2 species, diversity calculations would be meaningless.

For proper diversity analysis:
1. Download all 327 species (large download, ~10-50GB depending on area)
2. Or use a pre-processed diversity layer if available

We demonstrate diversity calculations in the advanced section below with more species.

## 7. Creating Maps

Generate publication-ready visualizations:

In [ ]:
# Create total biomass map
maps = api.create_maps(
    zarr_path="data/wake_county.zarr",
    map_type="biomass",  # Total biomass from downloaded species
    output_dir="maps/"
)

print(f"Created {len(maps)} map files:")
for m in maps:
    print(f"  {m}")

In [ ]:
# Create species maps
maps = api.create_maps(
    zarr_path="data/wake_county.zarr",
    map_type="species",
    species=["0131"],  # Loblolly pine
    output_dir="maps/",
    state="NC"  # Add state boundary overlay
)

print(f"Created species maps: {maps}")

### Display the Maps

If running in Jupyter/Colab, display the created images:

In [ ]:
from IPython.display import Image, display
from pathlib import Path

# Display any PNG files in the maps directory
maps_dir = Path("maps")
if maps_dir.exists():
    for png_file in maps_dir.glob("*.png"):
        print(f"\n{png_file.name}:")
        display(Image(filename=str(png_file), width=600))

## 8. Complete Example

Here's a complete workflow wrapped in a function:

In [ ]:
def analyze_county_biomass(state: str, county: str, species_codes: list):
    """Analyze biomass for selected species in a county."""
    from gridfia import GridFIA
    from pathlib import Path
    
    api = GridFIA()
    
    # Create safe directory name
    dir_name = f"{state.lower().replace(' ', '_')}_{county.lower().replace(' ', '_')}"
    data_dir = f"data/{dir_name}"
    
    print(f"Analyzing {county}, {state}...")
    print(f"Species: {species_codes}")
    
    # 1. Download species data
    print("\n1. Downloading species data...")
    files = api.download_species(
        output_dir=data_dir,
        species_codes=species_codes,
        state=state,
        county=county
    )
    print(f"   Downloaded {len(files)} files")
    
    # 2. Create Zarr store
    print("\n2. Creating Zarr store...")
    zarr_path = api.create_zarr(
        input_dir=data_dir,
        output_path=f"{data_dir}.zarr"
    )
    print(f"   Created: {zarr_path}")
    
    # 3. Calculate total biomass
    print("\n3. Calculating total biomass...")
    results = api.calculate_metrics(
        zarr_path=zarr_path,
        calculations=["total_biomass"]
    )
    
    # 4. Print summary
    print(f"\n{'='*50}")
    print(f"Biomass Analysis: {county}, {state}")
    print(f"{'='*50}")
    print(f"Species analyzed: {len(species_codes)}")
    
    for result in results:
        print(f"\n{result.name}:")
        if result.statistics:
            for stat, value in result.statistics.items():
                print(f"  {stat}: {value:.2f}")
    
    return results

# Uncomment to run (takes 5-15 minutes):
# results = analyze_county_biomass(
#     state="North Carolina",
#     county="Durham",
#     species_codes=["0131", "0316", "0802", "0833"]
# )

## 9. Tips and Best Practices

### Download Strategy
- Start with **county-level** analysis (faster downloads)
- Use specific **species codes** instead of downloading all 327 species
- State-level downloads can take 30+ minutes

### Memory Management
- Zarr stores enable lazy loading - data is only read when needed
- For very large areas, process in chunks

### Data Quality
- BIGMAP data is from 2018 - forests change over time
- NoData values indicate non-forest or areas outside the study region
- Verify CRS matches your other GIS layers (Web Mercator by default)

## 10. Next Steps

Now that you can create biomass maps, explore more:

### Advanced gridFIA
- Custom bounding boxes for specific areas
- Comparison maps showing multiple species
- Integration with GIS software via GeoTIFF export

### Other FIAtools
- **[pyFIA](https://fiatools.org/tools/pyfia/)**: Query plot-level FIA data
- **[pyFVS](https://fiatools.org/tools/pyfvs/)**: Simulate forest growth
- **[askFIA](https://fiatools.org/tools/askfia/)**: Natural language queries

### Resources
- [gridFIA Documentation](https://mihiarc.github.io/gridfia/)
- [BIGMAP Data Portal](https://apps.fs.usda.gov/arcx/rest/services/RDW_Biomass)
- [FIA Species Codes](https://www.fia.fs.usda.gov/library/field-guides-methods-proc/)

---

**Questions or feedback?** Open an issue on [GitHub](https://github.com/mihiarc/gridfia/issues).